In [1]:
import os
import json
import pandas as pd
from src.vycisti import vycisti
pd.set_option('display.max_columns', None)
df = pd.read_json(os.path.join("data", "filmy.json"))
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())
df = vycisti(df, opravy)

2023/06/01 17:14:46 z 4555 řádků odstraněno 133. Aktuální dataframe má 4422 řádků.


In [2]:
persony = pd.read_json(os.path.join("data", "persony.json")).drop_duplicates()

In [3]:
df = df.explode("Lokace")

In [4]:
def mesto(lokace):
    
    if isinstance(lokace, str):
    
        if "(" in lokace:

            zavorka = lokace.split("(")[1].replace(")","").strip()

            if zavorka in seznamy["praha"]:

                mesto = "Praha"

            elif zavorka == "Česká republika":
                
                mesto = lokace.split("(")[0].strip()
            
            else:

                mesto = zavorka

        else:
            
            mesto = lokace
            
        if "Česká republika, " in mesto:
                
            mesto = mesto.split(",")[1].strip()
            
    else:

        mesto = lokace         
        
    return(mesto)

In [5]:
df["Město"] = df["Lokace"].apply(lambda x: mesto(x))

## Nejčastější lokace

České celovečerní hrané filmy od r. 1998 včetně:

In [6]:
hcf = df[df["Délka"] == "dlouhometrážní"].explode("Typologie")
hcf = hcf[hcf["Typologie"] == "hraný"]
hcf = hcf[hcf["Copyright"] >= 1998]

Pořadí měst podle počtu natáčených filmů:

In [7]:
hcf.groupby("Město")["Film"].nunique().nlargest(60)

Město
Praha                          170
Slovensko                       52
Brno                            10
Jaroměř                          9
Bratislava                       8
Rakousko                         8
Beroun                           7
Kladno                           7
Zlín                             7
Hradec Králové                   6
Itálie                           6
Liberec                          6
Mladá Boleslav                   6
Olomouc                          6
Polsko                           6
Praha a okolí                    6
Vysočany                         6
Francie                          5
Německo                          5
Ostrava                          5
Plzeň                            5
Ústí nad Labem                   5
Brdy                             4
Doksy                            4
Jizerské hory                    4
Karlovy Vary                     4
Kolín                            4
Kouřim                           4
Mikulov       

In [34]:
top_mesta = hcf.groupby("Město")["Film"].nunique().nlargest(60)
top_mesta = top_mesta[top_mesta >= 4]
top_mesta = top_mesta.index.to_list()[2:]

Kompletní seznam filmů a lokací ve městech:

In [43]:
for t in top_mesta:
    print(f"{t.upper()}:")
    filmy = hcf[hcf["Město"] == t]["Film"].drop_duplicates().to_list()
    for f in filmy:
        print(f"""{f.upper()} | Lokality: {", ".join(hcf[(hcf["Město"] == t) & (hcf["Film"] == f)]["Lokace"].drop_duplicates().to_list())}""")
    print("***")

BRNO:
ZEJTRA NAPOŘÁD (400555) | Lokality: Brno (Česká republika)
LÍDA BAAROVÁ (400696) | Lokality: Brno (Česká republika), Národní divadlo – Mahenovo divadlo (Brno), Černá Pole (Brno), Pisárky (Brno)
VŠECHNO BUDE FAJN! (401386) | Lokality: Brno
PEPA (401760) | Lokality: Brno (Česká republika)
ALENKA V ZEMI ZÁZRAKŮ (401746) | Lokality: Brno (Česká republika), hlavní nádraží (Brno), Mendlovo náměstí (Brno), Opatství svatého Tomáše /Starobrněnský klášter/ (Brno), Jakubské náměstí (Brno), kostel sv. Jakuba Staršího (Brno), Petrov (Brno), katedrála sv. Petra a Pavla (Brno), Kraví hora (Brno), hvězdárna a planetárium (Brno), Výstaviště (Brno), Jihlavská ulice (Brno), krematorium (Brno), Ústřední hřbitov (Brno)
NABARVENÉ PTÁČE (401962) | Lokality: Česká republika, Brno (Česká republika), ulice Cejl (Brno)
SKLENĚNÝ POKOJ (401929) | Lokality: Česká republika, Brno (Česká republika), Výstaviště (Brno), Černá Pole (Brno), Vila Tugendhat (Brno), Pisárky (Brno), Vila Stiassni (Brno), Zelný trh (Brn

Místa v Praze:

In [9]:
hcf[hcf["Město"] == "Praha"].groupby("Lokace")["Film"].nunique().nlargest(50)

Lokace
Česká republika, Praha (Česká republika)    45
Nové Město (Praha)                          43
Vltava (Praha)                              29
Vinohrady (Praha)                           26
Žižkov (Praha)                              25
Staré Město (Praha)                         24
Malá Strana (Praha)                         23
Praha (Česká republika)                     20
Hradčany (Praha)                            19
Bubeneč (Praha)                             18
Holešovice (Praha)                          18
Smíchov (Praha)                             17
Dejvice (Praha)                             15
Nusle (Praha)                               15
Václavské náměstí (Nové Město)              15
Karlín (Praha)                              12
Josefov (Praha)                             11
Podolí (Praha)                              10
Praha, Vltava (Praha)                       10
Ruzyně (Praha)                               9
Mánesův most (Vltava)                        8
Perner

## Rodiště herců a hereček

Všechny filmy všech délek a typů.

In [10]:
top100 = df.explode("Hrají").groupby("Hrají")["Film"].nunique().nlargest(100).index.to_list()

In [11]:
top500 = df.explode("Hrají").groupby("Hrají")["Film"].nunique().nlargest(500).index.to_list()

Města podle toho, kolik se v nich narodilo ze 100 nejobsazovanějších herců a hereček:

In [12]:
persony[persony["Jméno"].isin(top100)].groupby("Město narození")["Jméno"].nunique().nlargest(10)

Město narození
Praha                         51
Plzeň                          5
Pelhřimov                      3
Jičín                          2
Kutná Hora                     2
Pardubice                      2
Adamov u Českých Budějovic     1
Bozkov u Semil                 1
Brno                           1
Buštěhrad u Kladna             1
Name: Jméno, dtype: int64

### Plzeňáci z top 100

In [13]:
persony[(persony["Jméno"].isin(top100) & (persony["Město narození"] == "Plzeň"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
4775,František Kovářík (126912),1886.0
5166,Růžena Šlemrová (127282),1886.0
21528,Jindřich Plachta (5829),1899.0
24356,František Kreuzmann (75624),1895.0
27480,Ota Motyčka (81341),1899.0


### Pelhřimováci z top 100

In [14]:
persony[(persony["Jméno"].isin(top100) & (persony["Město narození"] == "Pelhřimov"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
13821,Karel Hábl (27177),1936.0
19618,Jiří Lír (4926),1923.0
32334,Lubomír Lipský (9728),1923.0


Města podle toho, kolik se v nich narodilo z 500 nejobsazovanějších herců a hereček:

In [15]:
persony[persony["Jméno"].isin(top500)].groupby("Město narození")["Jméno"].nunique().nlargest(40)

Město narození
Praha                                 223
Plzeň                                  20
Brno                                   10
České Budějovice                        7
Zlín                                    6
Hradec Králové                          5
Kutná Hora                              4
Ostrava                                 4
Pardubice                               4
Třebíč                                  4
Kroměříž                                3
Liberec                                 3
Mladá Boleslav                          3
Pelhřimov                               3
Prostějov                               3
Příbram                                 3
Bělá pod Bezdězem                       2
Dvůr Králové nad Labem                  2
Havlíčkův Brod                          2
Holice                                  2
Horní Suchá u Karviné                   2
Hořice v Podkrkonoší                    2
Jihlava                                 2
Jičín              

In [16]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Nová Včelnice u Kamenice nad Lipou"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
7356,Luděk Munzar (12942),1933.0
19510,Rudolf Hrušínský st. (4842),1920.0


In [17]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Bělá pod Bezdězem"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
4274,Jan Maška (126431),1921.0
17111,Václav Trégl (35224),1902.0


In [18]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Hořice v Podkrkonoší"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
5376,Alois Dvorský (127491),1883.0
20244,Stanislav Fišer (5344),1931.0


In [19]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Holice"))][["Jméno","Rok narození"]][["Jméno","Rok narození"]]

,Jméno,Rok narození
1985,Jan Kačer (1165),1936.0
17533,Václav Lohniský (36488),1920.0


In [20]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Úvaly u Prahy"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
20685,Josef Střecha (55613),1907.0
26363,Oleg Reif (7964),1928.0


In [21]:
persony[(persony["Jméno"].isin(top500) & (persony["Město narození"] == "Horní Suchá u Karviné"))][["Jméno","Rok narození"]]

,Jméno,Rok narození
14641,Bronislav Poloczek (2877),1939.0
20102,Marie Glázrová (5247),1911.0


### Úmrtí

Místa úmrtí stovky nejobsazovanějších:

In [22]:
persony[persony["Jméno"].isin(top500)].groupby("Město úmrtí")["Jméno"].nunique().nlargest(40)

Město úmrtí
Praha                               338
Brno                                  5
Plzeň                                 5
Mělník                                3
Nová Ves pod Pleší                    3
Bratislava                            2
Olomouc                               2
Říčany u Prahy                        2
Bořanovice u Prahy                    1
Brandýs nad Labem                     1
Hamburk                               1
Hollenstedt                           1
Hostivice u Prahy                     1
Houštka u Mladé Boleslavi             1
Hradec Králové                        1
Jilemnice                             1
Kalifornie                            1
Karlovy Vary                          1
Kladno                                1
Kralupy nad Vltavou                   1
Kytín                                 1
Letovice u Blanska                    1
Liberec                               1
Libochovice u Roudnice nad Labem      1
Mladá Vožice u Tábora       

Kdo umřel v Nové vsi:

In [44]:
persony[(persony["Jméno"].isin(top500) & (persony["Město úmrtí"] == "Nová Ves pod Pleší"))][["Jméno","Rok úmrtí"]]

,Jméno,Rok úmrtí
797,Josef Somr (1054),2022.0
13887,Věra Kalendová (27289),2003.0
17893,Vladimír Brabec (3858),2017.0


## Rodiště režisérů

Stovka nejproduktivnějších režisérů:

In [45]:
top100r = df.explode("Režie").groupby("Režie")["Film"].nunique().nlargest(100).index.to_list()

In [46]:
persony[persony["Jméno"].isin(top100r)].groupby("Město narození")["Jméno"].nunique().nlargest(40)

Město narození
Praha                          41
Brno                            3
Kladno                          3
Plzeň                           3
Ostrava                         2
Prostějov                       2
České Budějovice                2
Český Těšín                     2
Berlín                          1
Bratislava                      1
Březové Hory u Příbrami         1
Chrudim                         1
Dobrošov u Náchoda              1
Dobřany                         1
Dolní Lukavice                  1
Dolní Štěpanice u Jilemnice     1
Hradec Králové                  1
Karlovy Vary                    1
Katovice u Strakonic            1
Kelč u Valašského Meziříčí      1
Kežmarok                        1
Ledeč nad Sázavou               1
Lomnice nad Popelkou            1
Lublaň                          1
Michalovce u Humenné            1
Mladá Boleslav                  1
Moravské Budějovice             1
Moskva                          1
Mělník                          1

In [47]:
persony[(persony["Jméno"].isin(top100r)) & (persony["Město narození"] == "Kladno")][["Jméno","Rok narození"]]

,Jméno,Rok narození
9840,Jiří Svoboda (1703),1945.0
19627,Zdenek Sirový (4935),1932.0
25975,Antonín Kachlík (7880),1923.0


In [48]:
persony[(persony["Jméno"].isin(top100r)) & (persony["Město narození"] == "Český Těšín")][["Jméno","Rok narození"]]

,Jméno,Rok narození
17355,Milan Cieslar (3593),1960.0
20011,František Vláčil (5191),1924.0
